# Implementando un sistema de recomendación con la librería Surprise de Python usando los algoritmos KNNBasic y KNNWithMean

# Fase 1 y Fase 2

Leer y replicar el ejercicio propuesto de la libreria Surprise en Python propuesto como ejercicio del presente proyecto.

Interpretar los resultados obtenidos en el notebook de trabajo, sobre los algoritmos utilizados y las métricas de regresión.

He decidido realizarlo todo el proceso de manera paralela, porque considero que con el código delante va a ser más fácil interpretar los resultados y las herramientas implementadas para conseguir llegar a esos resultados.

Para este artículo debemos explicar varios elementos que vamos a poner en práctica en el desarrollo del proyecto.

## ¿Qué es un sistema de recomendación?¿Principales SR a conocer por parte del lector?


Un **SR** es un sistema automatizado para sugerir items relevantes para un usuario basado en sus preferencias, comportamiento o historial de interacción con el sistema. 

Principales tipos descritos brevemente serían:

- **Basado en popularidad**: Recomienda los ítems más populares entre todos los usuarios sin tener en cuenta sus preferencias individuales

- **Basado en contenido**: utilizan características de los ítems, para encontrar ítems similares y recomendarlos al usuario.

- **Híbrido**: Combinan diversos enfoques de recomendación, como la popularidad y el contenido.

- **Colaborativo**: Emplean las interacciones y las opiniones de otros usuarios para hacer recomendaciones personalizadas a un usuario. Pudiendo implementar 2 tipos de filtros. **User-item filtering**: se toma a un usuario, se encuentran usuarios similares y se recomiendas ítems que a esos usuarios similares le gustaron. Por otro lado, **item-item filtering**: se toma un item, se encuentran usuarios que les haya gustado ese ítem, y se buscan otros ítems que a esos usuarios les haya gustado.

## ¿Qué es Surprise?

**Surprise** es una librería de Python enfocada en los sistemas de recomendación, nos permite tratar datos explícitos (expresamente mencionados o proporcionados de manera clara y directa). Realizando filtrados colaborativos (utiliza la información sobre las interacciones previas de los usuarios con los items para recomendar nuevos items relevantes para el usuario seleccionado). 

Está diseñada para ser fácil de usar, flexible, poder trabajar con diferentes tipos de datos y modelos de recomendación.


## Puesta en práctica

Pedimos que se descague e instale a través del gestor de paquetes de python: **pip**

In [1]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366485 sha256=40e8d8a0ee736b1413b8d1e6399ff64deb5d2bc3f6fe283d3c816e81823aab55
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


Hemos importado las librerías correspondientes, entre ella de las que nos vamos a ayudar principalmente para este Sistema de recomendación **Surprise**.

**KNNBasic** y **KNNWithMeans**: Dos algoritmos de recomendación basados en KNN dentro de la librería **Surprise**.

**Dataset**: Clase de la librería **Surprise** para almacenar y modificar los datos específicos para llevar a cabo los sistemas de recomendación con esta librería.

**Reader**: Clase de la librería **Surprise** para leer los datos y prepararlos para llevar a cabo los sistemas de recomendación con esta librería.

**Accuracy**: Función de la librería **Surprise** para evaluar la precisión de nuestros sistemas de recomendación.

**Train_test_split**: Función de la librería **Surprise** para dividir el conjunto de los datos en conjunto de entrenamiento y conjunto de test.

In [4]:
import pandas as pd
from surprise import KNNBasic, KNNWithMeans
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Trabajaremos con los datos de MovieLens (**ratings.dat**).

Utilizamos la clase **Reader** (leer datos y prepararlos), le indicamos **line_format** (formato de las líneas en el archivo, le indicamos que la primera columna corresponde a user, posteriormente item, rating y por último timestamp), le indicamos también **sep**, donde los valores se separan a partir de '::'. 

Posteriormente usamos la clase **Dataset** (almacenar datos y modificarlos), donde cargamos el archivo ratings.dat de la ruta de mi drive en un objeto "data" usando el reader de la clase Reader para indicarle cómo debe leer y preparar los datos.

In [5]:
reader = Reader(line_format='user item rating timestamp', sep='::',skip_lines=1)
data = Dataset.load_from_file('/content/drive/MyDrive/Glosario herramientas para un Data Science/IEBS/Módulo 14. Sistemas de recomendación/Clase 4/ratings.dat', reader=reader)

Se divide data a través de **train_test_split**, en un conjunto de entrenamiento al que le asignamos el 70% de los datos y un conjunto de test al que le asignamos el 30% de los datos.

In [6]:
train, test = train_test_split(data, test_size=0.3)

**KNNBasic** y **KNNWithMeans**: Dos algoritmos de recomendación basados en KNN dentro de la librería **Surprise** donde **KNNBasic** está basado en KNN para hacer las recomendaciones y por otro lado, **KNNWithMeans** se una modificación de **KNNBasic**, pero en este caso se implementa la media de las calificaciones de los vecinos para intentar mejorar las recomendaciones.

Para el objeto knn y kMeans utilizamos el algoritmo **KNNBasic** y **KNNWithMeans** donde los requerimientos que solicitamos son que el parámetro **k** correspondiente al número de vecinos más cercanos para la realización de la búsqueda 50 (ambos algoritmos, en el primer caso para user y en el segundo para items) y por otro lado **sim_options** para indicarle al algoritmo a través de este diccionario la medida de similitud a establecer, en el caso del primer algoritmo "pearson" y para el segundo "cosine". Por último **user_based** para indicarle al algoritmo que criterio debe seguir basándose en usuarios o en items, en el primer algoritmo le indicamos que se base en usuarios, mientras que en el segundo le indicamos que se base en items.

Formula pearson

$$r = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^{n} (x_i - \bar{x})^2} \sqrt{\sum_{i=1}^{n} (y_i - \bar{y})^2}}$$


Formula similitud del coseno

$$similitud(x,y) = \frac{\sum_{i=1}^{n} x_i y_i}{\sqrt{\sum_{i=1}^{n} x_i^2} \sqrt{\sum_{i=1}^{n} y_i^2}}
$$

In [7]:
knn = KNNBasic(k=50, sim_options={'name': 'pearson', 'user_based': True})
kMeans = KNNWithMeans(k=50, sim_options={'name': 'cosine','user_based': False})

Entrenamos los modelos que hemos creado con el método **fit**  y le suministramos el conjunto de entrenamiento para ello. 

Podemos apreciar que nos devuelve un mensaje indicándonos que para el primer modelo lo esta computando con la medida de similitud de pearson y posteriormente que se ha completado el cálculo de la matriz de similitud. De manera similar para el segundo modelo pero en este caso con la medida de similitud coseno. 

Por ultimo la ubicación de la clase en la biblioteca y la dirección de memoria única.

In [8]:
knn.fit(train)
kMeans.fit(train)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


Voy a realizar un ejemplo a mano para el usuario 42, vemos que para item 7, no existe puntuación, que para item 3354 la puntuación es 3, para el item 648 la puntuación fue 4 y para el ítem 2628 la puntuación fue 5.

In [18]:
df = pd.read_csv('/content/drive/MyDrive/Glosario herramientas para un Data Science/IEBS/Módulo 14. Sistemas de recomendación/Clase 4/ratings.dat', sep='::')
df.columns = ['user', 'item', 'puntuacion','momento']

In [21]:
df.loc[df['user'] == 42]

,user,item,puntuacion,momento
5702,42,2989,3,978041044
5703,42,3421,4,978041747
5704,42,648,4,978041143
5705,42,3354,3,978040451
5706,42,2628,5,978039063
...,...,...,...,...
5928,42,1240,5,978040211
5929,42,2116,4,978039266
5930,42,2117,3,978039659
5931,42,2985,5,978040259


In [20]:
df.loc[(df['user'] == 42) & (df['item'] == 7)]

,user,item,puntuacion,momento


Tenemos los modelos entrenados, en este caso realizamos un ejemplo donde seleccionamos el **user_id**: 42 (usuario con el nº id 42) y **item_id**: 7 (item con el identificador 7). Llamamos al método **predict** y la predicción de la calificación que el user 42 dispondrá sobre el item 7 se almacenan en **knn_user_prediction** y  **kMeans_user_prediction**.

In [22]:
user_id = str(42)
item_id = str(7)
knn_user_prediction = knn.predict(user_id, item_id)
kMeans_user_prediction = kMeans.predict(user_id, item_id)
print(knn_user_prediction)
print(kMeans_user_prediction)

user: 42         item: 7          r_ui = None   est = 3.64   {'actual_k': 50, 'was_impossible': False}
user: 42         item: 7          r_ui = None   est = 3.37   {'actual_k': 50, 'was_impossible': False}


Para item 3354 la puntuación es 3, para el item 648 la puntuación fue 4 y para el ítem 2628 la puntuación fue 5. Podemos ver en esta serie de ejemplos las estimaciones. Para estos casos al existir una relación de puntuación real entre user e ítem, la representamos en r_ui con su valor asociado.

In [27]:
user_id = str(42)
item_id = str(3354)
knn_user_prediction = knn.predict(user_id, item_id, r_ui=3)
kMeans_user_prediction = kMeans.predict(user_id, item_id, r_ui=3)
print(knn_user_prediction)
print(kMeans_user_prediction)

user: 42         item: 3354       r_ui = 3.00   est = 2.64   {'actual_k': 50, 'was_impossible': False}
user: 42         item: 3354       r_ui = 3.00   est = 2.76   {'actual_k': 50, 'was_impossible': False}


In [28]:
user_id = str(42)
item_id = str(648)
knn_user_prediction = knn.predict(user_id, item_id, r_ui=4)
kMeans_user_prediction = kMeans.predict(user_id, item_id, r_ui=4)
print(knn_user_prediction)
print(kMeans_user_prediction)

user: 42         item: 648        r_ui = 4.00   est = 3.34   {'actual_k': 50, 'was_impossible': False}
user: 42         item: 648        r_ui = 4.00   est = 3.44   {'actual_k': 50, 'was_impossible': False}


In [29]:
user_id = str(42)
item_id = str(2628)
knn_user_prediction = knn.predict(user_id, item_id, r_ui=5)
kMeans_user_prediction = kMeans.predict(user_id, item_id, r_ui=5)
print(knn_user_prediction)
print(kMeans_user_prediction)

user: 42         item: 2628       r_ui = 5.00   est = 4.33   {'actual_k': 50, 'was_impossible': False}
user: 42         item: 2628       r_ui = 5.00   est = 3.61   {'actual_k': 50, 'was_impossible': False}


En este caso, llevamos a cabo las predicciones con **knn.test** y **kMeans.test** sobre el conjunto de test y se almacenan en **knn_test_predictions** y **kMeans_test_predictions**. A continuación evaluamos las precisiones de las predicciones implementando **accuracy.rmse** para que nos calcule el error cuadrático medio de ambos algoritmos y almacenar los resultados en **knn_rmse** y **kMeans_rmse**.

In [31]:
knn_test_predictions = knn.test(test)
kMeans_test_predictions = kMeans.test(test)
knn_rmse = accuracy.rmse(knn_test_predictions)
kMeans_rmse = accuracy.rmse(kMeans_test_predictions)
print('KNN RMSE: ' + str(knn_rmse))
print('KMeans RMSE: ' + str(kMeans_rmse))

RMSE: 0.9640
RMSE: 0.8983
KNN RMSE: 0.9640204768914648
KMeans RMSE: 0.8982688105231328


# Conclusiones

## Resultados

$$ RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y_i})^2} $$

**RMSE** es la raíz cuadrada de la media de los errores al cuadrado entre los valores reales y los valores predecidos. Cuanto más bajo sea el RMSE, mejor será la precisión del modelo. En vista de los resultados podríamos concluir que el modelo KMeans es un modelo que minimiza en mayor medida el error(0.8982).

## Posibles modificaciones

Hemos seleccionado una serie de hiperparámetros como son k=50 (Voy a hacerlo para k=30 y k=70)

La mejora de hiperparámetros debería realizarla fuera de **Surprise**, pero como solo voy a poner 2 casos (uno por encima y otro por debajo a modo de ejemplo no llevará demasiado tiempo). Se podría haber realizado un GridSearch para varios de los hiperparámetros para asegurarnos que nos estamos acercando al mejor modelo posible.

En los 2 casos aquí mencionados no mejoró el planteado en el apartado de los resultados.

In [ ]:
knn = KNNBasic(k=30, sim_options={'name': 'pearson', 'user_based': True})
kMeans = KNNWithMeans(k=30, sim_options={'name': 'cosine','user_based': False})
knn.fit(train)
kMeans.fit(train)
knn_test_predictions = knn.test(test)
kMeans_test_predictions = kMeans.test(test)
knn_rmse = accuracy.rmse(knn_test_predictions)
kMeans_rmse = accuracy.rmse(kMeans_test_predictions)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9698
RMSE: 0.8995


In [30]:
knn = KNNBasic(k=70, sim_options={'name': 'pearson', 'user_based': True})
kMeans = KNNWithMeans(k=70, sim_options={'name': 'cosine','user_based': False})
knn.fit(train)
kMeans.fit(train)
knn_test_predictions = knn.test(test)
kMeans_test_predictions = kMeans.test(test)
knn_rmse = accuracy.rmse(knn_test_predictions)
kMeans_rmse = accuracy.rmse(kMeans_test_predictions)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9640
RMSE: 0.8983
